# CONNECT

In [7]:
# You can enable debugging by adding this piece of code to the top of your project

import logging
logging.basicConfig(level=logging.DEBUG)

In [8]:
# core의 경로를 sys.path에 추가

import sys
sys.path.append('/Users/jeongyunl/Documents/GitHub/wikibaseIntegrator_address_test2/core')
print(sys.path)

['/Users/jeongyunl/Documents/GitHub/wikibaseIntegrator_address_test2', '/Users/jeongyunl/opt/anaconda3/envs/wikibase/lib/python39.zip', '/Users/jeongyunl/opt/anaconda3/envs/wikibase/lib/python3.9', '/Users/jeongyunl/opt/anaconda3/envs/wikibase/lib/python3.9/lib-dynload', '', '/Users/jeongyunl/opt/anaconda3/envs/wikibase/lib/python3.9/site-packages', '/Users/jeongyunl/Documents/GitHub/wikibaseIntegrator_address_test2/core']


In [9]:
from wikibaseintegrator import WikibaseIntegrator as WI
from wikibaseintegrator.wbi_config import config as wbi_config

wbi_config['MEDIAWIKI_API_URL'] = 'http://localhost:4100/w/api.php'
wbi_config['SPARQL_ENDPOINT_URL'] = 'http://localhost:8834/proxy/wdqs/bigdata/namespace/wdq/sparql'
wbi_config['WIKIBASE_URL'] = 'http://wikibase.svc'
wbi_config['USER_AGENT'] = 'MyWikibaseBot/1.0 (https://www.wikidata.org/wiki/User:JeongYunLee)'

In [10]:
from wikibaseintegrator import wbi_login

login_instance = wbi_login.Login(user='JeongYunLee', password='naby753951', mediawiki_api_url='http://localhost:4100/w/api.php')

DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): localhost:4100
DEBUG:urllib3.connectionpool:http://localhost:4100 "POST /w/api.php HTTP/1.1" 200 111
DEBUG:urllib3.connectionpool:http://localhost:4100 "POST /w/api.php HTTP/1.1" 200 323
INFO:wikibaseintegrator.wbi_login:Successfully logged in as JeongYunLee
DEBUG:urllib3.connectionpool:http://localhost:4100 "GET /w/api.php?action=query&meta=tokens&type=csrf&format=json HTTP/1.1" 200 110


In [11]:
import pywikibot, json, csv, sys
from pywikibot import family

# ADD PROPERTIES

In [8]:
# 임포트 할 데이터 형태 확인하기

import csv 

csv_path = '/Users/jeongyunl/Documents/GitHub/wikibaseIntegrator_address_test2/testData/add_properties.csv'
csv_file = open(csv_path,'r', encoding='utf-8-sig')

csv_reader = csv.DictReader(csv_file)

for row in csv_reader:
    row = dict(row)
    print(row)

    for key in row:
        print(key)

{'Property Label': '식별자', 'Datatype': 'string', 'Property Description': 'identifier'}
Property Label
Datatype
Property Description
{'Property Label': '구분', 'Datatype': 'string', 'Property Description': 'index'}
Property Label
Datatype
Property Description
{'Property Label': '용어출처', 'Datatype': 'string', 'Property Description': 'source of term'}
Property Label
Datatype
Property Description
{'Property Label': '용어(영문)', 'Datatype': 'string', 'Property Description': 'term(eng)'}
Property Label
Datatype
Property Description
{'Property Label': '용어(로마자)', 'Datatype': 'string', 'Property Description': 'term(rom)'}
Property Label
Datatype
Property Description
{'Property Label': '용어(한자)', 'Datatype': 'string', 'Property Description': 'term(zh)'}
Property Label
Datatype
Property Description
{'Property Label': '별칭 또는 줄임말', 'Datatype': 'string', 'Property Description': 'another name or abbreviation'}
Property Label
Datatype
Property Description
{'Property Label': '최종출처', 'Datatype': 'string', 'Prop

In [9]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

"""
    Notes:
    We need to remove the built in throttling because we 
    are working on our own localhost running wikibase, we don't
    care if we do a ton of requests, we are likely the only user
"""
# over write it
def wait(self, seconds):
    """Wait for seconds seconds.
    Announce the delay if it exceeds a preset limit.
    """
    pass
    # if seconds <= 0:
    #     return

    # message = (u"Sleeping for %(seconds).1f seconds, %(now)s" % {
    #     'seconds': seconds,
    #     'now': time.strftime("%Y-%m-%d %H:%M:%S",
    #                          time.localtime())
    # })
    # if seconds > config.noisysleep:
    #     pywikibot.output(message)
    # else:
    #     pywikibot.log(message)

    
    # time.sleep(seconds)

pywikibot.throttle.Throttle.wait = wait

if __name__ == "__main__":


    # if len(sys.argv) == 1:
    #     raise ValueError('Missing input CSV file')

    # csv_path = sys.argv[1]
    # csv_file = open(csv_path,'r')

    # csv_reader = csv.DictReader(csv_file)
    csv_path = '/Users/jeongyunl/Documents/GitHub/wikibaseIntegrator_address_test2/testData/add_properties.csv'
    csv_file = open(csv_path,'r', encoding='utf-8-sig')
    csv_reader = csv.DictReader(csv_file)

    # # If you changed the name of the site to something else make sure to change it here
    site = pywikibot.Site('mediawiki:mediawiki')
    #site.login()

    complete_data = []

    for row in csv_reader:
        row = dict(row)
        

        # build our API data call for each property
        data = {
            'datatype': row['Datatype'],  # mandatory
            'descriptions': {
                'ko': {
                    'language': 'ko',
                    'value': row['Property Description']
                }
            },
            'labels': {
                'ko': {
                    'language': 'ko',
                    'value': row['Property Label']
                }
            }
        }

        params = {
            'action': 'wbeditentity',
            'new': 'property',
            'data': json.dumps(data),
            'summary': 'bot adding in properties',
            'token': site.tokens['csrf']
            # 'token': site.tokens['edit']
        }

        req = site._simple_request(**params)
        results = req.submit()

        row['PID'] = results['entity']['id']

        print(row['PID'], row['Property Label'])
        complete_data.append(row)

    csv_file.close()

    with open(csv_path+'_updated.csv','w') as out:

        fieldnames = list(complete_data[0].keys())
        writer = csv.DictWriter(out, fieldnames=fieldnames)

        writer.writeheader()
        writer.writerows(complete_data)


  req = site._simple_request(**params)



P50 식별자
P51 구분
P52 용어출처
P53 용어(영문)
P54 용어(로마자)
P55 용어(한자)
P56 별칭 또는 줄임말
P57 최종출처
P58 등록일


# ADD CORE ITEMS

In [5]:
# 임포트 할 데이터 형태 확인하기

import csv 

csv_path = '/Users/jeongyunl/Documents/Github/wikibaseintegrator_address_test2/testData/add_core_items.csv'
csv_file = open(csv_path,'r', encoding='utf-8')
csv_reader = csv.DictReader(csv_file)

for row in csv_reader:
    row = dict(row)
    print(row)

    for key in row:
        print(key)

{'Label': 'Address Terms', 'Description': 'An address terminology dictionary project by Hike Lab.'}
Label
Description


In [7]:
#### data-2-wikibase guthub

import wikidataintegrator as WI
import requests, csv, sys, re


"""
    NOTES:
    The wikidataintegrator module was built to do a sepcific task, to populate 
    wikdiata with Genes, Proteins, Diseases, Drugs and other domain specific stuff
    so we need to overwrite a few pieces of it in order to make it work for our domain spcific stuff
    and also work on our own installation of wikibase not wikidata
"""

# overwrite the property store that comes built in with wikidataintergrator
# these are unique values that should be unique in our data
# if you have spefifc properties that should be unique across your data you can add them here
# it is useful because then you can use that property to pull up the item without knowing its QID number
# WI.wdi_property_store.wd_properties = {
#     'P6': {
#         'datatype': 'string',
#         'name': 'LJ Slug ID', 
#         'domain': ['linkedjazz'], # this is a wikidataintergrator thing, to group properties together
#         'core_id': True 
#     }
# }

### 원래 위에 코드인데 wdi_property_store가 없다고 에러가 뜸
### 그래서 아래 코드로 바꿈
### https://github.com/SuLab/WikidataIntegrator/blob/master/wikidataintegrator/wdi_property_store.py 파일 삭제된 듯
WI.wdi_core.WDItemEngine.core_props = { 
    'P8': {
        'datatype': 'string',
        'name': 'LJ Slug ID', 
        'domain': ['linkedjazz'], # this is a wikidataintergrator thing, to group properties together
        'core_id': True 
    }
}

# use this specific property for this run of the script
use_unique_property = 'P8:string'
# Change the pwd paramater here 
# bot_user='Admin'
# bot_pwd='bot@4vvaepj4quu1ahbmporh1ujk9qh0pqd4'

# it uses some methods that need to have the default sparql endpoint changed from the wikidata address
# but it uses them in the modules with the defaults, so lets over write the method with our own with the our host as the default
# def execute_sparql_query(prefix='', query='', endpoint='http://localhost:8282/proxy/wdqs/bigdata/namespace/wdq/sparql',
#                          user_agent="User Agent Name"):
#     """
#     Static method which can be used to execute any SPARQL query
#     :param prefix: The URI prefixes required for an endpoint, default is the Wikidata specific prefixes
#     :param query: The actual SPARQL query string
#     :param endpoint: The URL string for the SPARQL endpoint. Default is the URL for the Wikidata SPARQL endpoint
#     :param user_agent: Set a user agent string for the HTTP header to let the WDQS know who you are.
#     :type user_agent: str
#     :return: The results of the query are returned in JSON format
#     """

#     # standard prefixes for the Wikidata SPARQL endpoint
#     # PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#> has been removed for performance reasons
#     # wd_standard_prefix = '''
#     #     PREFIX wd: <http://www.wikidata.org/entity/>
#     #     PREFIX wdt: <http://www.wikidata.org/prop/direct/>
#     #     PREFIX wikibase: <http://wikiba.se/ontology#>
#     #     PREFIX p: <http://www.wikidata.org/prop/>
#     #     PREFIX v: <http://www.wikidata.org/prop/statement/>
#     #     PREFIX q: <http://www.wikidata.org/prop/qualifier/>
#     #     PREFIX ps: <http://www.wikidata.org/prop/statement/>

#     # '''

#     # we are not doing the standard prefixes for localhost

#     # if prefix == '':
#     #     prefix = wd_standard_prefix

#     # -_-.... some escaping quotes here...
#     quoute_check = re.search("\s'(.*)'\s.", query)
#     if quoute_check is not None:
#         replace_with = quoute_check[1].replace("'","\\'")
#         query = query.replace(quoute_check[1],replace_with)


#     params = {
#         'query': prefix + '\n#Tool: PBB_core fastrun\n' + query,
#         'format': 'json'
#     }

#     headers = {
#         'Accept': 'application/sparql-results+json',
#         'User-Agent': user_agent
#     }
#     response = requests.get(endpoint, params=params, headers=headers)
#     response.raise_for_status()

#     return response.json()


# WI.wdi_core.WDItemEngine.execute_sparql_query = execute_sparql_query


if __name__ == "__main__":


    # if len(sys.argv) == 1:
    #     raise ValueError('Missing input CSV file')
    csv_path = '/Users/jeongyunl/Documents/Github/wikibaseintegrator_address_test2/testData/add_core_items.csv'
    csv_file = open(csv_path,'r', encoding='utf-8-sig')
    csv_reader = csv.DictReader(csv_file)

    # login_instance = WI.wdi_login.WDLogin(user=bot_user, pwd=bot_pwd, server="localhost:8181", base_url_template='http://{}/w/api.php')

    # we are going to just create the entities and any P values provided in the CSV
    complete_data = []
    errors_data = []

    for row in csv_reader:
        row = dict(row)

        data = []


        # properties stuff     
        for key in row:

            # is it a property field
            if key.find(':') > -1 and key[0] == "P":
                PID, data_type = key.split(':')

                # there are few other data types need to add...
                try:
                    if data_type.lower() == 'string':
                        if row[key] is not None and row[key].strip() != '':
                            statement = WI.wdi_core.WDString(value=row[key], prop_nr=PID)
                            data.append(statement)
                    elif data_type.lower() == 'url':
                        if row[key] is not None and row[key].strip() != '':
                            statement = WI.wdi_core.WDUrl(value=row[key], prop_nr=PID)
                            data.append(statement)
                    elif data_type.lower() == 'wikibase-item':
                        if row[key] is not None and row[key].strip() != '':
                            statement = WI.wdi_core.WDItemID(value=row[key], prop_nr=PID)
                            data.append(statement)

                    
                except Exception as e:
                    print("There was an error with this part1, skipping:")
                    print(row)
                    print(e)
                    errors_data.append(row)
                    data = "error"

        if data == 'error':
            continue


        # change this to whatever field you are using to be unique across the data
        # item_name = None if row['Label'] is None or row['Label'].strip() == '' else row['Label']
        # here im using this P35 property, a legacy identifier that will be unique across the dataset
        if use_unique_property in row:
            item_name = row[use_unique_property]
        else:
            item_name = None

        # what domain is this data going into, can be None
        domain = 'linkedjazz'

        try:
            # instead of item_name you can use "item_id" and pass the QID if you have that
            # wd_item = WI.wdi_core.WDItemEngine(item_name=item_name, domain=domain, data=data, server="localhost:4100", base_url_template='http://localhost:4100/w/api.php')
            ### 위에 코드 에러나서 아래와 같이 바꿈 (참고: https://github.com/xjsachs/flora-commons-wikibase/blob/2222de9fddacf607132dd474ea775a1f8971cd4f/flora-commons-data-import/add_items.py)
            wd_item = WI.wdi_core.WDItemEngine(new_item=True, data=data, mediawiki_api_url="http://localhost:4100/w/api.php")

            # set the label and description if exists
            if 'Label' in row:
                if row['Label'] is not None and row['Label'].strip() != '':
                    wd_item.set_label(row['Label'])

            if 'Description' in row:
                if row['Description'] is not None and row['Description'].strip() != '':
                    wd_item.set_description(row['Description'])
        
            # can update instead and tell it which properties to append to not overwrite
            # wd_item.update(data,append_value=['P17'])

            # write
            r = wd_item.write(login_instance)
            print("goodbye!")
            
            # QID is returned
            row['QID'] = r

            print(row['QID'], row['Label'])

            
            complete_data.append(row)


        except Exception as e:
            print("There was an error with this part2, skipping:")
            print(row)
            print(e)
            errors_data.append(row)
            data = "error"


    csv_file.close()

    with open(csv_path+'_updated.csv','w') as out:

        fieldnames = list(complete_data[0].keys())
        writer = csv.DictWriter(out, fieldnames=fieldnames)

        writer.writeheader()
        writer.writerows(complete_data)


    if len(errors_data) > 0:
        with open(csv_path+'_errors.csv','w') as out:

            fieldnames = list(errors_data[0].keys())
            writer = csv.DictWriter(out, fieldnames=fieldnames)

            writer.writeheader()
            writer.writerows(errors_data)





goodbye!
Q992 Address Terms


# ADD ITEMS

In [11]:
# 임포트 할 데이터 형태 확인하기

import csv 

csv_path = '/Users/jeongyunl/Documents/Github/wikibaseintegrator_address_test2/testData/add_items_sample.csv'
csv_file = open(csv_path,'r', encoding='utf-8')
csv_reader = csv.DictReader(csv_file)

for row in csv_reader:
    row = dict(row)
    print(row)

    for key in row:
        print(key)

{'Label': '가로등', 'P50:string': '1', 'P51:string': 'ㄱ', 'P52:string': '행안부주소용어목록', 'P53:string': 'street light', 'P54:string': 'garodeung', 'P55:string': '街路燈', 'P56:string': '', 'Description': '거리의 조명이나 교통의 안전, 또는 미관 따위를 위하여 길가를 따라 설치해 놓은 등', 'P57:string': '표준국어대사전', 'P58:string': ''}
Label
P50:string
P51:string
P52:string
P53:string
P54:string
P55:string
P56:string
Description
P57:string
P58:string
{'Label': '가상주소', 'P50:string': '2', 'P51:string': 'ㄱ', 'P52:string': '행안부주소용어목록', 'P53:string': 'virtual address', 'P54:string': 'gasangjuso', 'P55:string': '假想住所', 'P56:string': '', 'Description': '', 'P57:string': '', 'P58:string': ''}
Label
P50:string
P51:string
P52:string
P53:string
P54:string
P55:string
P56:string
Description
P57:string
P58:string
{'Label': '각각출입구', 'P50:string': '3', 'P51:string': 'ㄱ', 'P52:string': '행안부주소용어목록', 'P53:string': '', 'P54:string': '', 'P55:string': '', 'P56:string': '', 'Description': '', 'P57:string': '', 'P58:string': ''}
Label
P50:string
P51:string
P5

In [12]:
#### data-2-wikibase guthub

# import wikidataintegrator as WI
import requests, csv, sys, re


"""
    NOTES:
    The wikidataintegrator module was built to do a sepcific task, to populate 
    wikdiata with Genes, Proteins, Diseases, Drugs and other domain specific stuff
    so we need to overwrite a few pieces of it in order to make it work for our domain spcific stuff
    and also work on our own installation of wikibase not wikidata
"""

# overwrite the property store that comes built in with wikidataintergrator
# these are unique values that should be unique in our data
# if you have spefifc properties that should be unique across your data you can add them here
# it is useful because then you can use that property to pull up the item without knowing its QID number
# WI.wdi_property_store.wd_properties = {
#     'P6': {
#         'datatype': 'string',
#         'name': 'LJ Slug ID', 
#         'domain': ['linkedjazz'], # this is a wikidataintergrator thing, to group properties together
#         'core_id': True 
#     }
# }

### 원래 위에 코드인데 wdi_property_store가 없다고 에러가 뜸
### 그래서 아래 코드로 바꿈
### https://github.com/SuLab/WikidataIntegrator/blob/master/wikidataintegrator/wdi_property_store.py 파일 삭제된 듯
WI.wdi_core.WDItemEngine.core_props = { 
    'P50': {
        'datatype': 'string',
        'name': '식별자', 
        'domain': ['addresTerms'], # this is a wikidataintergrator thing, to group properties together
        'core_id': True 
    }
}

# use this specific property for this run of the script
use_unique_property = 'P50:string'
# Change the pwd paramater here 
# bot_user='Admin'
# bot_pwd='bot@4vvaepj4quu1ahbmporh1ujk9qh0pqd4'

# it uses some methods that need to have the default sparql endpoint changed from the wikidata address
# but it uses them in the modules with the defaults, so lets over write the method with our own with the our host as the default
# def execute_sparql_query(prefix='', query='', endpoint='http://localhost:8282/proxy/wdqs/bigdata/namespace/wdq/sparql',
#                          user_agent="User Agent Name"):
#     """
#     Static method which can be used to execute any SPARQL query
#     :param prefix: The URI prefixes required for an endpoint, default is the Wikidata specific prefixes
#     :param query: The actual SPARQL query string
#     :param endpoint: The URL string for the SPARQL endpoint. Default is the URL for the Wikidata SPARQL endpoint
#     :param user_agent: Set a user agent string for the HTTP header to let the WDQS know who you are.
#     :type user_agent: str
#     :return: The results of the query are returned in JSON format
#     """

#     # standard prefixes for the Wikidata SPARQL endpoint
#     # PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#> has been removed for performance reasons
#     # wd_standard_prefix = '''
#     #     PREFIX wd: <http://www.wikidata.org/entity/>
#     #     PREFIX wdt: <http://www.wikidata.org/prop/direct/>
#     #     PREFIX wikibase: <http://wikiba.se/ontology#>
#     #     PREFIX p: <http://www.wikidata.org/prop/>
#     #     PREFIX v: <http://www.wikidata.org/prop/statement/>
#     #     PREFIX q: <http://www.wikidata.org/prop/qualifier/>
#     #     PREFIX ps: <http://www.wikidata.org/prop/statement/>

#     # '''

#     # we are not doing the standard prefixes for localhost

#     # if prefix == '':
#     #     prefix = wd_standard_prefix

#     # -_-.... some escaping quotes here...
#     quoute_check = re.search("\s'(.*)'\s.", query)
#     if quoute_check is not None:
#         replace_with = quoute_check[1].replace("'","\\'")
#         query = query.replace(quoute_check[1],replace_with)


#     params = {
#         'query': prefix + '\n#Tool: PBB_core fastrun\n' + query,
#         'format': 'json'
#     }

#     headers = {
#         'Accept': 'application/sparql-results+json',
#         'User-Agent': user_agent
#     }
#     response = requests.get(endpoint, params=params, headers=headers)
#     response.raise_for_status()

#     return response.json()


# WI.wdi_core.WDItemEngine.execute_sparql_query = execute_sparql_query


if __name__ == "__main__":


    # if len(sys.argv) == 1:
    #     raise ValueError('Missing input CSV file')

    csv_path = '/Users/jeongyunl/Documents/Github/wikibaseintegrator_address_test2/testData/add_items_sample2.csv'
    csv_file = open(csv_path,'r', encoding='utf-8-sig')
    csv_reader = csv.DictReader(csv_file)

    # login_instance = WI.wdi_login.WDLogin(user=bot_user, pwd=bot_pwd, server="localhost:8181", base_url_template='http://{}/w/api.php')

    # we are going to just create the entities and any P values provided in the CSV
    complete_data = []
    errors_data = []

    for row in csv_reader:
        row = dict(row)

        data = []


        # properties stuff     
        for key in row:

            # is it a property field
            if key.find(':') > -1 and key[0] == "P":
                PID, data_type = key.split(':')

                # there are few other data types need to add...
                try:
                    if data_type.lower() == 'string':
                        if row[key] is not None and row[key].strip() != '':
                            statement = WI.wdi_core.WDString(value=row[key], prop_nr=PID)
                            data.append(statement)
                    elif data_type.lower() == 'url':
                        if row[key] is not None and row[key].strip() != '':
                            statement = WI.wdi_core.WDUrl(value=row[key], prop_nr=PID)
                            data.append(statement)
                    elif data_type.lower() == 'wikibase-item':
                        if row[key] is not None and row[key].strip() != '':
                            statement = WI.wdi_core.WDItemID(value=row[key], prop_nr=PID)
                            data.append(statement)

                    
                except Exception as e:
                    print("There was an error with this part1, skipping:")
                    print(row)
                    print(e)
                    errors_data.append(row)
                    data = "error"

        if data == 'error':
            continue


        # change this to whatever field you are using to be unique across the data
        # item_name = None if row['Label'] is None or row['Label'].strip() == '' else row['Label']
        # here im using this P35 property, a legacy identifier that will be unique across the dataset
        # if use_unique_property in row:
        #     item_name = row[use_unique_property]
        #     print('item_name=', item_name)
        # else:
        #     item_name = None
        #     print('none')

        # what domain is this data going into, can be None
        domain = 'addresTerms'

        try:
            # instead of item_name you can use "item_id" and pass the QID if you have that
            # wd_item = WI.wdi_core.WDItemEngine(item_name=item_name, domain=domain, data=data, server="localhost:4100", base_url_template='http://localhost:4100/w/api.php')
            ### 위에 코드 에러나서 아래와 같이 바꿈 (참고: https://github.com/xjsachs/flora-commons-wikibase/blob/2222de9fddacf607132dd474ea775a1f8971cd4f/flora-commons-data-import/add_items.py)
            wd_item = WI.wdi_core.WDItemEngine(new_item=True, data=data, mediawiki_api_url="http://localhost:4100/w/api.php")

            # set the label and description if exists
            if 'Label' in row:
                if row['Label'] is not None and row['Label'].strip() != '':
                    wd_item.set_label(row['Label'])

            if 'Description' in row:
                if row['Description'] is not None and row['Description'].strip() != '':
                    wd_item.set_description(row['Description'])
        
            # can update instead and tell it which properties to append to not overwrite
            # wd_item.update(data,append_value=['P17'])

            # write
            r = wd_item.write(login_instance)
            print("goodbye!")
            
            # QID is returned
            row['QID'] = r

            print(row['QID'], row['Label'])

            
            complete_data.append(row)


        except Exception as e:
            print("There was an error with this part2, skipping:")
            print(row)
            print(e)
            errors_data.append(row)
            data = "error"


    csv_file.close()

    with open(csv_path+'_updated.csv','w') as out:

        fieldnames = list(complete_data[0].keys())
        writer = csv.DictWriter(out, fieldnames=fieldnames)

        writer.writeheader()
        writer.writerows(complete_data)


    if len(errors_data) > 0:
        with open(csv_path+'_errors.csv','w') as out:

            fieldnames = list(errors_data[0].keys())
            writer = csv.DictWriter(out, fieldnames=fieldnames)

            writer.writeheader()
            writer.writerows(errors_data)





goodbye!
Q993 가로등
goodbye!
Q994 가상주소
goodbye!
Q995 각각출입구
goodbye!
Q996 개방자치단체코드
goodbye!
Q997 개방형 출입구
goodbye!
Q998 갱신일자
goodbye!
Q999 거주건물
goodbye!
Q1000 건물
goodbye!
Q1001 건물 경계
goodbye!
Q1002 건물군
goodbye!
Q1003 건물군일련번호
goodbye!
Q1004 건물등
goodbye!
Q1005 건물명
goodbye!
Q1006 건물명칭
goodbye!
Q1007 건물번호
goodbye!
Q1008 건물번호판
goodbye!
Q1009 건물본번
goodbye!
Q1010 건물부번
goodbye!
Q1011 건물시설
goodbye!
Q1012 건물종속구분
goodbye!
Q1013 격자방식
goodbye!
Q1014 경로유형
goodbye!
Q1015 공간객체 구축
goodbye!
Q1016 공간참조체계
goodbye!
Q1017 공중연결경로
goodbye!
Q1018 관리번호
goodbye!
Q1019 관심지정정보
goodbye!
Q1020 교량일련번호
goodbye!
Q1021 구역방식
goodbye!
Q1022 구역번호
goodbye!
Q1023 국가지점번호판 설치 의무지역
goodbye!
Q1024 국가지점번호판 점검
goodbye!
Q1025 국가지점번호판 확충
goodbye!
Q1026 궤적
goodbye!
Q1027 기초구역관리번호
goodbye!
Q1028 기초번호간격
goodbye!
Q1029 기초번호방식
goodbye!
Q1030 기초번호판 확충
goodbye!
Q1031 기타구역
goodbye!
Q1032 내비게이션용유
goodbye!
Q1033 농로
goodbye!
Q1034 농촌주소
goodbye!
Q1035 다방향출입구
goodbye!
Q1036 다중단방향출입구
goodbye!
Q1037 단방향출입구
goodbye!
Q1038 단지 내 도로
goodbye!
Q1039 도로구간데이터

KeyboardInterrupt: 

## add items sample2

In [2]:
# 임포트 할 데이터 형태 확인하기

import csv 

csv_path = '/Users/jeongyunl/Documents/Github/wikibaseintegrator_address_test2/testData/add_items_sample2.csv'
csv_file = open(csv_path,'r', encoding='utf-8')
csv_reader = csv.DictReader(csv_file)

for row in csv_reader:
    row = dict(row)
    print(row)

    for key in row:
        print(key)

{'Label': '법정동', 'P5': 'Q992', 'P50:string': '184', 'P51:string': 'ㅂ', 'P52:string': '행안부주소용어목록', 'P53:string': 'Statutory Neighborhood', 'P54:string': 'Beopjeong-dong', 'P55:string': '法定洞', 'P56:string': '', 'Description': '법으로 지정된 동이란 뜻으로, 정부의 토지 구획, 지적도, 재산권 행사와 같은 법률 행위시 일반적으로 주소로 사용되는 동 명칭이다. 도로명주소에서 괄호 안에 병기하는 동의 명칭이나 부동산 등기 등 부동산 관련 토지표기에 쓰이며, 변동이 제한적으로 이루어진다.', 'P57:string': 'TTA 정보통신용어사전', 'P58:string': None}
Label
P5
P50:string
P51:string
P52:string
P53:string
P54:string
P55:string
P56:string
Description
P57:string
P58:string
{'Label': '보행자길', 'P5': 'Q992', 'P50:string': '191', 'P51:string': 'ㅂ', 'P52:string': '행안부주소용어목록', 'P53:string': 'Pedestrian exclusive use road', 'P54:string': 'bohaengjagil', 'P55:string': '步行者專用道路', 'P56:string': '보행자전용도로', 'Description': '보행자만 다닐 수 있도록 안전표지나 그와 비슷한 인공구조물로 표시한 도로를 말한다', 'P57:string': '법령용어사전', 'P58:string': None}
Label
P5
P50:string
P51:string
P52:string
P53:string
P54:string
P55:string
P56:string
Description
P57:string
P58:string


In [24]:
#### data-2-wikibase guthub

import wikidataintegrator as WI
import requests, csv, sys, re


"""
    NOTES:
    The wikidataintegrator module was built to do a sepcific task, to populate 
    wikdiata with Genes, Proteins, Diseases, Drugs and other domain specific stuff
    so we need to overwrite a few pieces of it in order to make it work for our domain spcific stuff
    and also work on our own installation of wikibase not wikidata
"""

# overwrite the property store that comes built in with wikidataintergrator
# these are unique values that should be unique in our data
# if you have spefifc properties that should be unique across your data you can add them here
# it is useful because then you can use that property to pull up the item without knowing its QID number
# WI.wdi_property_store.wd_properties = {
#     'P6': {
#         'datatype': 'string',
#         'name': 'LJ Slug ID', 
#         'domain': ['linkedjazz'], # this is a wikidataintergrator thing, to group properties together
#         'core_id': True 
#     }
# }

### 원래 위에 코드인데 wdi_property_store가 없다고 에러가 뜸
### 그래서 아래 코드로 바꿈
### https://github.com/SuLab/WikidataIntegrator/blob/master/wikidataintegrator/wdi_property_store.py 파일 삭제된 듯
WI.wdi_core.WDItemEngine.core_props = { 
    'P50': {
        'datatype': 'string',
        'name': '식별자', 
        'domain': ['addresTerms'], # this is a wikidataintergrator thing, to group properties together
        'core_id': True 
    }
}

# use this specific property for this run of the script
use_unique_property = 'P50:string'
# Change the pwd paramater here 
# bot_user='Admin'
# bot_pwd='bot@4vvaepj4quu1ahbmporh1ujk9qh0pqd4'

# it uses some methods that need to have the default sparql endpoint changed from the wikidata address
# but it uses them in the modules with the defaults, so lets over write the method with our own with the our host as the default
# def execute_sparql_query(prefix='', query='', endpoint='http://localhost:8282/proxy/wdqs/bigdata/namespace/wdq/sparql',
#                          user_agent="User Agent Name"):
#     """
#     Static method which can be used to execute any SPARQL query
#     :param prefix: The URI prefixes required for an endpoint, default is the Wikidata specific prefixes
#     :param query: The actual SPARQL query string
#     :param endpoint: The URL string for the SPARQL endpoint. Default is the URL for the Wikidata SPARQL endpoint
#     :param user_agent: Set a user agent string for the HTTP header to let the WDQS know who you are.
#     :type user_agent: str
#     :return: The results of the query are returned in JSON format
#     """

#     # standard prefixes for the Wikidata SPARQL endpoint
#     # PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#> has been removed for performance reasons
#     # wd_standard_prefix = '''
#     #     PREFIX wd: <http://www.wikidata.org/entity/>
#     #     PREFIX wdt: <http://www.wikidata.org/prop/direct/>
#     #     PREFIX wikibase: <http://wikiba.se/ontology#>
#     #     PREFIX p: <http://www.wikidata.org/prop/>
#     #     PREFIX v: <http://www.wikidata.org/prop/statement/>
#     #     PREFIX q: <http://www.wikidata.org/prop/qualifier/>
#     #     PREFIX ps: <http://www.wikidata.org/prop/statement/>

#     # '''

#     # we are not doing the standard prefixes for localhost

#     # if prefix == '':
#     #     prefix = wd_standard_prefix

#     # -_-.... some escaping quotes here...
#     quoute_check = re.search("\s'(.*)'\s.", query)
#     if quoute_check is not None:
#         replace_with = quoute_check[1].replace("'","\\'")
#         query = query.replace(quoute_check[1],replace_with)


#     params = {
#         'query': prefix + '\n#Tool: PBB_core fastrun\n' + query,
#         'format': 'json'
#     }

#     headers = {
#         'Accept': 'application/sparql-results+json',
#         'User-Agent': user_agent
#     }
#     response = requests.get(endpoint, params=params, headers=headers)
#     response.raise_for_status()

#     return response.json()


# WI.wdi_core.WDItemEngine.execute_sparql_query = execute_sparql_query


if __name__ == "__main__":


    # if len(sys.argv) == 1:
    #     raise ValueError('Missing input CSV file')

    csv_path = '/Users/jeongyunl/Documents/Github/wikibaseintegrator_address_test2/testData/add_items_sample2.csv'
    csv_file = open(csv_path,'r', encoding='utf-8-sig')
    csv_reader = csv.DictReader(csv_file)

    # login_instance = WI.wdi_login.WDLogin(user=bot_user, pwd=bot_pwd, server="localhost:8181", base_url_template='http://{}/w/api.php')

    # we are going to just create the entities and any P values provided in the CSV
    complete_data = []
    errors_data = []

    for row in csv_reader:
        row = dict(row)

        data = []


        # properties stuff     
        for key in row:

            # is it a property field
            if key.find(':') > -1 and key[0] == "P":
                PID, data_type = key.split(':')

                # there are few other data types need to add...
                try:
                    if data_type.lower() == 'string':
                        if row[key] is not None and row[key].strip() != '':
                            statement = WI.wdi_core.WDString(value=row[key], prop_nr=PID)
                            data.append(statement)
                    elif data_type.lower() == 'url':
                        if row[key] is not None and row[key].strip() != '':
                            statement = WI.wdi_core.WDUrl(value=row[key], prop_nr=PID)
                            data.append(statement)
                    elif data_type.lower() == 'wikibase-item':
                        if row[key] is not None and row[key].strip() != '':
                            statement = WI.wdi_core.WDItemID(value=row[key], prop_nr=PID)
                            data.append(statement)

                    
                except Exception as e:
                    print("There was an error with this part1, skipping:")
                    print(row)
                    print(e)
                    errors_data.append(row)
                    data = "error"

        if data == 'error':
            continue


        # change this to whatever field you are using to be unique across the data
        # item_name = None if row['Label'] is None or row['Label'].strip() == '' else row['Label']
        # here im using this P35 property, a legacy identifier that will be unique across the dataset
        # if use_unique_property in row:
        #     item_name = row[use_unique_property]
        #     print('item_name=', item_name)
        # else:
        #     item_name = None
        #     print('none')

        # what domain is this data going into, can be None
        domain = 'addresTerms'

        try:
            # instead of item_name you can use "item_id" and pass the QID if you have that
            # wd_item = WI.wdi_core.WDItemEngine(item_name=item_name, domain=domain, data=data, server="localhost:4100", base_url_template='http://localhost:4100/w/api.php')
            ### 위에 코드 에러나서 아래와 같이 바꿈 (참고: https://github.com/xjsachs/flora-commons-wikibase/blob/2222de9fddacf607132dd474ea775a1f8971cd4f/flora-commons-data-import/add_items.py)
            wd_item = WI.wdi_core.WDItemEngine(new_item=True, data=data, mediawiki_api_url="http://localhost:4100/w/api.php")

            ### 이 부분에서 언어 설정, 추가해주면 제일 위에 박스에 추가할 수 있음
            # set the label and description if exists
            if 'Label' in row:
                if row['Label'] is not None and row['Label'].strip() != '':
                    wd_item.set_label(row['Label'], lang='ko')
                    
            if 'Label_eng' in row:
                if row['Label_eng'] is not None and row['Label_eng'].strip() != '':
                    wd_item.set_label(row['Label_eng'], lang='en')
                    
            if 'Label_zh' in row:
                if row['Label_zh'] is not None and row['Label_zh'].strip() != '':
                    wd_item.set_label(row['Label_zh'], lang='zh')
                else:
                    pass

            if 'Description' in row:
                if row['Description'] is not None and row['Description'].strip() != '':
                    wd_item.set_description(row['Description'], lang='ko')
        
            # can update instead and tell it which properties to append to not overwrite
            # wd_item.update(data,append_value=['P17'])

            # write
            r = wd_item.write(login_instance)
            print("goodbye!")
            
            # QID is returned
            row['QID'] = r

            print(row['QID'], row['Label'])

            
            complete_data.append(row)


        except Exception as e:
            print("There was an error with this part2, skipping:")
            print(row)
            print(e)
            errors_data.append(row)
            data = "error"


    csv_file.close()

    with open(csv_path+'_updated.csv','w') as out:

        fieldnames = list(complete_data[0].keys())
        writer = csv.DictWriter(out, fieldnames=fieldnames)

        writer.writeheader()
        writer.writerows(complete_data)


    if len(errors_data) > 0:
        with open(csv_path+'_errors.csv','w') as out:

            fieldnames = list(errors_data[0].keys())
            writer = csv.DictWriter(out, fieldnames=fieldnames)

            writer.writeheader()
            writer.writerows(errors_data)



DEBUG:urllib3.connectionpool:Resetting dropped connection: localhost
DEBUG:urllib3.connectionpool:http://localhost:4100 "POST /w/api.php HTTP/1.1" 200 1028


goodbye!
Q1093 법정동


DEBUG:urllib3.connectionpool:http://localhost:4100 "POST /w/api.php HTTP/1.1" 200 969


goodbye!
Q1094 보행자길
